In [1]:
import ROOT

Welcome to JupyROOT 6.18/04


In [3]:
import fastjet as fj
import fjext
import fjcontrib

import pythia8
import pythiafjext
import pythiaext

In [4]:
# Generator.
pythia = pythia8.Pythia()
extra_s = ["Next:numberCount = 0", "Next:numberShowEvent = 0", "Next:numberShowInfo = 0", "Next:numberShowProcess = 0", "Stat:showProcessLevel = on"]
for s in extra_s:
    pythia.readString(s)
# Allow no substructure in e+- beams: normal for corrected LEP data.
pythia.readString("PDF:lepton = off")
# Process selection.
pythia.readString("WeakSingleBoson:ffbar2gmZ = on")
# Switch off all Z0 decays and then switch back on those to quarks.
pythia.readString("23:onMode = off");
pythia.readString("23:onIfAny = 1 2 3 4 5")
# LEP1 initialization at Z0 mass.
pythia.readString("Beams:idA =  11")
pythia.readString("Beams:idB = -11")
mZ = pythia.particleData.m0(23)
pythia.settings.parm("Beams:eCM", mZ)
# parton level first
pythia.readString("HadronLevel:all=off")
pythia.init()

True


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.244      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 20 Dec 2019  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 01 Jul 2021 at 10:41:42    |  | 
 |  |                                                                              |  | 
 |  |   Christian Bi

In [5]:
# print the banner first
fj.ClusterSequence.print_banner()
print()
# set up our jet definition and a jet selector
jet_R0 = 1.0
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
print(jet_def)


Longitudinally invariant anti-kt algorithm with R = 1 and E scheme recombination
#--------------------------------------------------------------------------
#                         FastJet release 3.3.3
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code
# and 3rd party plugin jet algorithms. See COPYING file for details.
#----------------------------------------------------------------------

In [6]:
# hadron level acceptamce
max_eta_hadron = 10.
from pyjetty.mputils import pwarning
pwarning('max eta for particles after hadronization set to', max_eta_hadron)
parts_selector_h = fj.SelectorAbsEtaMax(max_eta_hadron)
jet_selector = fj.SelectorPtMin(1.0) & fj.SelectorPtMax(100.0) & fj.SelectorAbsEtaMax(max_eta_hadron - 1.05 * jet_R0)

max_eta_parton = max_eta_hadron + 3. * jet_R0
pwarning('max eta for partons set to', max_eta_parton)
parts_selector_p = fj.SelectorAbsEtaMax(max_eta_parton)


[w] max eta for particles after hadronization set to 10.0
[w] max eta for partons set to 13.0


In [7]:
#output
outf = ROOT.TFile('lep_pythia8_out.root', 'recreate')
outf.cd()
t = ROOT.TTree('t', 't')
from pyjetty.mputils import RTreeWriter
tw = RTreeWriter(tree=t)
hpt_parton = ROOT.TH1F('pt_parton_level', 'pt_parton_level', 50, 0, 50)
hpt_hadron = ROOT.TH1F('pt_hadron_level', 'pt_hadron_level', 50, 0, 50)
hpt_hadron.SetLineColor(3)

In [8]:
# event loop
nev = 10000
# import tqdm
from tqdm.notebook import tqdm
for iev in tqdm(range(nev)):
    if not pythia.next():
        continue

    #select particles
    parts_pythia_p = pythiafjext.vectorize_select(pythia, [pythiafjext.kFinal], 0, True)
    parts_pythia_p_selected = parts_selector_p(parts_pythia_p)

    hstatus = pythia.forceHadronLevel()
    if not hstatus:
        pwarning('forceHadronLevel false event', iev)
        continue
    # parts_pythia_h = pythiafjext.vectorize_select(pythia, [pythiafjext.kHadron, pythiafjext.kCharged])
    parts_pythia_h = pythiafjext.vectorize_select(pythia, [pythiafjext.kFinal], 0, True)
    parts_pythia_h_selected = parts_selector_h(parts_pythia_h)

    parts_pythia_hch = pythiafjext.vectorize_select(pythia, [pythiafjext.kFinal, pythiafjext.kCharged], 0, True)
    parts_pythia_hch_selected = parts_selector_h(parts_pythia_hch)

    # jet finder is just an extra - not needed for generation
    jets_p = fj.sorted_by_pt(jet_def(parts_pythia_p))
    jets_h = fj.sorted_by_pt(jet_def(parts_pythia_h))
    jets_ch_h = fj.sorted_by_pt(jet_selector(jet_def(parts_pythia_hch)))

    _c = [hpt_parton.Fill(j.perp()) for j in jets_p]
    _c = [hpt_hadron.Fill(j.perp()) for j in jets_h]


 PYTHIA Error in StringFragmentation::fragment: stuck in joining  
 PYTHIA Error in Pythia::forceHadronLevel: hadronLevel failed; try again  


In [9]:
pythia.stat()


 *-------  PYTHIA Event and Cross Section Statistics  -------------------------------------------------------------*
 |                                                                                                                 |
 | Subprocess                                    Code |            Number of events       |      sigma +- delta    |
 |                                                    |       Tried   Selected   Accepted |     (estimated) (mb)   |
 |                                                    |                                   |                        |
 |-----------------------------------------------------------------------------------------------------------------|
 |                                                    |                                   |                        |
 | f fbar -> gamma*/Z0                            221 |       10520      10000      10000 |   4.146e-05  2.084e-13 |
 |                                                    |        

In [10]:
%jsroot on
# js has to work within lab or notebook...
# run root --notebook ...
canvas = ROOT.TCanvas("cpt","cpt",400,400)
hpt_parton.Draw()
hpt_hadron.Draw('same')
canvas.SetLogy()
canvas.BuildLegend()
canvas.Draw()

In [11]:
outf.Write()
outf.Close()